### GET TEMPLATES CONTAINING > 2 NOUNS

### GET DICTIONARIES POSTAG --> WORDS

### GET DICTIONARIES WORDS --> POSTAG


In [1]:
import nltk
import keras.preprocessing
import codecs
import re
import pickle
import random

c:\users\hp\anaconda2\envs\translator\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with codecs.open('data/frost/input.txt','rb', encoding='utf-8') as f:
    data=f.read()

In [3]:
#CLEAN TEXT
def simple_clean(string):
        string = re.sub(r"\n", " KKK ", string) #MARK LINES
        string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)
        string = re.sub(r"\'", "", string)
        string = re.sub(r"-", "", string)
        string = re.sub(r":", "", string)

        string = re.sub(r"\'s", " \'s", string)
        string = re.sub(r"\'d", "ed", string) #for the old style
        string = re.sub(r"n\'t", " n\'t", string)
        string = re.sub(r"I\'ve", "I have", string)
        string = re.sub(r"\'ll", " will", string)

        string = re.sub(r"[0-9]+", "EOS", string) # EOS tag for numeric titling

        string = re.sub(r";", ",", string)
        string = re.sub(r",", " , ", string)
        string = re.sub(r"!", " ! ", string)
        string = re.sub(r"\?", " ? ", string)
        string = re.sub(r"\.", " . ", string)

        string = re.sub(r"\(", " \( ", string)
        string = re.sub(r"\)", " \) ", string)
        string = re.sub(r"\s{2,}", " ", string)

        return string.strip().split('KKK')

In [4]:
#GET TOKENIZED LINES
def get_token_sent(data):
    k=simple_clean(data)
    result=[]
    for el in k:
        res=el.split(' ')
        if set(res)!=set(['']):
            res=[x for x in res if x!='']
            #res=['sos']+res+['eos']
            result.append(res)
    return result

#GET POSTAG FOR SENTENCE
def get_pos_tag(sent):
    res=nltk.pos_tag(sent)
    res=[tup[1] for tup in res]
    return res

In [5]:
#GET TOKENIZED SENTENCE FOR THE TEXT
dat=get_token_sent(data)

In [6]:
#POSTAG ALL LINES
i=0
dataset=[]
while True:
    try:
        r=dat[i]
    except:
        break
    tag=get_pos_tag(r)
    if (tag.count('NN')+tag.count('NNS'))>1:
        dataset.append(tag)
    i+=1

In [7]:
# HOW MANY SINGULAR NOUNS WITH HAVE IN LINE 2?
get_pos_tag(dat[1]).count('NN')

0

In [8]:
#fly is a noun according to this, but it should also be a verb
nltk.pos_tag(['fly'])

[('fly', 'NN')]

In [9]:
# In order to get all possible tags for a particular word, we need to search over some corpus and find them.
#Search all tags for every word in the dataset
from collections import defaultdict, Counter
import itertools
counts = defaultdict(Counter)
tagged_sents = [nltk.pos_tag(sent) for sent in dat]
for word, pos in itertools.chain(*tagged_sents):
     counts[word][pos] += 1

In [10]:
from collections import defaultdict, Counter
import itertools
counts_sent= defaultdict(int)
for sent in dataset:
    str_sent=' '.join(sent)
    counts_sent[str_sent] += 1

In [11]:
#fly is a verb and a noun
counts['fly']

Counter({'NN': 13, 'VB': 6})

In [12]:
#kind is a noun, but 
print (counts['report'])
print(counts['kind'])

##add more examples pbt or brown nltk.corpus.brown

Counter()
Counter({'NN': 16})


In [13]:
# search over brown corpus too
"""from nltk.corpus import brown
print (brown.words())
tagged_sents = nltk.pos_tag(brown.words())
for word, pos in tagged_sents:
     counts[word][pos] += 1"""

'from nltk.corpus import brown\nprint (brown.words())\ntagged_sents = nltk.pos_tag(brown.words())\nfor word, pos in tagged_sents:\n     counts[word][pos] += 1'

In [14]:
#create dictionary TAG--> WORDS
tag2words=defaultdict(list)
wordTOtags=defaultdict(list)
word2tags=defaultdict(list)
for word, tag in zip(list(counts.keys()), list(counts.values())):
    mc=tag.most_common(3)
    for pos_t in mc:
        #tag2words[pos_t].append(word.lower())
        wordTOtags[word.lower()].append(pos_t)
for word, tag in zip(list(wordTOtags.keys()), list(wordTOtags.values())):
    list_t=sorted(tag, key=lambda tup: tup[1], reverse=True)
    for x in list_t:
        if float(x[1])/(list_t[0][1])>0.3:
            word2tags[word].append(x[0])
            tag2words[x[0]].append(word)
            

In [15]:
list(counts.values())[192].most_common(2)

[('DT', 70), ('NNP', 19)]

In [16]:
word2tags['was']

['VBD']

In [17]:
tag2words['NNS'][:10]

['wishes',
 'trees',
 'walls',
 'raspberries',
 'fences',
 'vines',
 'woods',
 'chops',
 'forgotten',
 'bats']

In [18]:
' '.join(dataset[0])

'CD IN PRP$ NNS VBZ IN DT JJ NNS ,'

In [19]:
NN2postag=defaultdict(list)
for sent in dataset:
    if sent.count('NN')>1:
        aux=[i for i,x in enumerate(sent) if x == 'NN']
        aux=random.sample(aux, 2)
        NN2postag['NN-NN'].append((sent, aux))
    if sent.count('NN')==1 and sent.count('NNS')==1:
        idxnn=sent.index('NN')
        idxnns=sent.index('NNS')
        NN2postag['NN-NNS'].append((sent,[idxnn, idxnns]))
    if sent.count('NNS')>1:
        aux=[i for i,x in enumerate(sent) if x == 'NNS']
        aux=random.sample(aux, 2)
        NN2postag['NNS-NNS'].append((sent, aux))
    #####################################
    if sent.count('VB')>1:
        aux=[i for i,x in enumerate(sent) if x == 'VB']
        aux=random.sample(aux, 2)
        NN2postag['VB-VB'].append((sent, aux))
    if sent.count('VB')>1:
        aux=[i for i,x in enumerate(sent) if x == 'VB']
        aux=random.sample(aux, 2)
        NN2postag['VB-VB'].append((sent, aux))
    if sent.count('NN')==1 and sent.count('VB')==1:
        idxnn=sent.index('NN')
        idxnns=sent.index('VB')
        NN2postag['NN-VB'].append((sent,[idxnn, idxnns]))
    if sent.count('NNS')==1 and sent.count('VB')==1:
        idxnn=sent.index('NNS')
        idxnns=sent.index('VB')
        NN2postag['NNS-VB'].append((sent,[idxnn, idxnns]))
    ##################################################
    if sent.count('RB')>1:
        aux=[i for i,x in enumerate(sent) if x == 'RB']
        aux=random.sample(aux, 2)
        NN2postag['RB-RB'].append((sent, aux))
    if sent.count('NN')==1 and sent.count('RB')==1:
        idxnn=sent.index('NN')
        idxnns=sent.index('RB')
        NN2postag['NN-RB'].append((sent,[idxnn, idxnns]))
    if sent.count('NNS')==1 and sent.count('RB')==1:
        idxnn=sent.index('NNS')
        idxnns=sent.index('RB')
        NN2postag['NNS-RB'].append((sent,[idxnn, idxnns]))
    if sent.count('RB')==1 and sent.count('VB')==1:
        idxnn=sent.index('RB')
        idxnns=sent.index('VB')
        NN2postag['RB-VB'].append((sent,[idxnn, idxnns]))
    #####################################################
    if sent.count('JJ')>1:
        aux=[i for i,x in enumerate(sent) if x == 'JJ']
        aux=random.sample(aux, 2)
        NN2postag['JJ-JJ'].append((sent, aux))
    if sent.count('NN')==1 and sent.count('JJ')==1:
        idxnn=sent.index('JJ')
        idxnns=sent.index('NN')
        NN2postag['JJ-NN'].append((sent,[idxnn, idxnns]))
    if sent.count('NNS')==1 and sent.count('JJ')==1:
        idxnn=sent.index('JJ')
        idxnns=sent.index('NNS')
        NN2postag['JJ-NNS'].append((sent,[idxnn, idxnns]))
    if sent.count('JJ')==1 and sent.count('VB')==1:
        idxnn=sent.index('JJ')
        idxnns=sent.index('VB')
        NN2postag['JJ-VB'].append((sent,[idxnn, idxnns]))
    if sent.count('JJ')==1 and sent.count('RB')==1:
        idxnn=sent.index('JJ')
        idxnns=sent.index('RB')
        NN2postag['JJ-RB'].append((sent,[idxnn, idxnns]))

In [27]:
tagTowords={}
for key, value in tag2words.items():
    tagTowords[key]=list(set(value))

In [96]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [30]:
#save results
with open('postag_dict.p', 'wb') as f:
    pickle.dump([NN2postag, tagTowords,word2tags],f, protocol=2 )

In [88]:
len(NN2postag['JJ-VB'])

184

In [89]:
NN2postag['NN-NNS'][0]

(['CC', 'VBD', 'DT', 'NN', 'CC', 'DT', 'JJ', 'NNS', ','], [3, 7])

In [90]:
len(NN2postag['NN-NNS'])

714

In [91]:
len(dataset)

3721